# Import your Libraries

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import requests
import os
import lusid
import cocoon
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()
scope = 'my_test_scope_8'

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.2885.0


# 1) Create Portfolios

## a) Import portfolio data

In [2]:
positions_file = pd.read_csv('./data/Sample_Positions.csv')
positions_file.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [3]:
portfolios_data = pd.DataFrame(positions_file['FundCode'])
portfolios_data['display_name'] = portfolios_data['FundCode'].apply(lambda x: "Fund {}".format(x))
portfolios_data['created'] = "2010-10-09T08:00:00Z"
portfolios_data['base_currency'] = 'GBP'
portfolios_data['description'] = portfolios_data['FundCode'].apply(lambda x: "The fund with the fund code {}".format(x))
portfolios_data.head()

,FundCode,display_name,created,base_currency,description
0,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
1,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
2,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
3,Portfolio-X,Fund Portfolio-X,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-X
4,Portfolio-X,Fund Portfolio-X,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-X


## c) Set your portfolio field mapping

In [4]:
portfolio_mapping_required = {
  'code': 'FundCode',
  'display_name': 'display_name',
  'created': 'created',
  'base_currency': 'base_currency'
}

portfolio_mapping_optional = {
  'description': 'description',
  'accounting_method': None
}

## d) Create your portfolios

In [5]:
responses = cocoon.create_portfolios_if_not_exist(
    client=client, 
    scope=scope, 
    data_frame=portfolios_data, 
    required_mapping=portfolio_mapping_required, 
    optional_mapping=portfolio_mapping_optional)

for portfolio_code, response in responses.items():
    prettyprint.portfolio_response(response)

# 2) Load your instrument universe

## a) Import instrument data

In [6]:
positions_file

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [7]:
instruments_data = pd.DataFrame(
    positions_file.loc[:, [
        'Local Currency Code', 
        'Security Description',
        'ISIN Security Identifier',
        'SEDOL Security Identifier']]
    )

instruments_data

,Local Currency Code,Security Description,ISIN Security Identifier,SEDOL Security Identifier
0,GBP,Lloyds Banking Group PLC,GB0008706128,870612
1,USD,Apple Inc,US0378331005,0865985
2,USD,Amazon,US0231351067,906866
3,GBX,Sainsbury,XS1139087933,B019KW7
4,USD,Tesla Inc,US88160R1014,BSJC712


## c) Set your instruments field mapping

In [8]:
instrument_mapping_required = {
  'name': 'Security Description'
}

instrument_identifier_mapping = {
  'identifier_mapping': {
    'ClientInternal': 'SEDOL Security Identifier',
    'Sedol': 'SEDOL Security Identifier',
    'Isin': 'ISIN Security Identifier'
  }
}

instrument_mapping_optional = {
  'definition': None
}

## d) Upsert your instruments

In [9]:
response = cocoon.upsert_instruments(
    client=client, 
    data_frame=instruments_data, 
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_mapping_required=instrument_mapping_required,
    instrument_mapping_optional=instrument_mapping_optional)

prettyprint.instrument_response(response)

Instrument Successfully Upserted: Amazon
ClientInternal ID: 906866
LUSID Instrument ID: LUID_RB16TCZW


Instrument Successfully Upserted: Tesla Inc
ClientInternal ID: BSJC712
LUSID Instrument ID: LUID_S9BYYAL3


Instrument Successfully Upserted: Lloyds Banking Group PLC
ClientInternal ID: 870612
LUSID Instrument ID: LUID_USQX4UWN


Instrument Successfully Upserted: Apple Inc
ClientInternal ID: 0865985 
LUSID Instrument ID: LUID_4Q2T9W41


Instrument Successfully Upserted: Sainsbury
ClientInternal ID: B019KW7
LUSID Instrument ID: LUID_UWJVIUUD


5  instruments upserted successfully
0  instrument upsert failures


# 3) Set your holdings

## a) Import holdings data

In [10]:
holdings_file = positions_file.copy(deep=True)
holdings_file

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [11]:
holdings_file['is_cash_with_currency'] = np.NaN
holdings_file.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency,is_cash_with_currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP,NaN
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD,NaN
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD,NaN
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP,NaN
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD,NaN


## c) Set your holdings field mapping

In [12]:
holding_mapping_required = {
  'portfolio_code': 'FundCode',
  'effective_date': 'Effective Date',
  'tax_lots.units': 'Quantity'
}

holding_identifier_mapping = {
  'identifier_mapping': {
      'Isin': 'ISIN Security Identifier',
      'Sedol': 'SEDOL Security Identifier'
  },
  'is_cash_with_currency': 'is_cash_with_currency'
}

holding_mapping_optional= {
  'tax_lots.cost.amount': None,
  'tax_lots.cost.currency': None,
  'tax_lots.portfolio_cost': None,
  'tax_lots.price': None,
  'tax_lots.purchase_date': None,
  'tax_lots.settlement_date': None
}

## d) Resolve to your instrument master

In [13]:
holdings_file = cocoon.resolve_instruments(
    client=client,
    data_frame=holdings_file,
    identifier_mapping=holding_identifier_mapping)

current_time = datetime.now(pytz.UTC)

holdings_file.loc[
    holdings_file['resolvable'] == False].to_csv(
        './data/UnResolved{}-{}-{}-{}-{}-{}.csv'.format(
            'Holdings',
            current_time.year,
            current_time.month,
            current_time.day,
            current_time.hour,
            current_time.minute))

holdings_file['resolvable'].value_counts()

Up to row 0


True    5
Name: resolvable, dtype: int64

## e) Add your holdings

In [14]:
holdings_file = holdings_file.loc[holdings_file['resolvable'] == True]

for effective_date in holdings_file['Effective Date'].unique():
    
    holdings_file_single_effective = holdings_file.loc[
        holdings_file['Effective Date'] == effective_date]

    responses = cocoon.load_file_multiple_portfolios(
        client=client, 
        scope=scope, 
        data_frame=holdings_file_single_effective,
        mapping_required=holding_mapping_required, 
        mapping_optional=holding_mapping_optional,
        source='Client',
        file_type='holding')
    
    for portfolio_code, response in responses.items():
        print ('\n')
        prettyprint.adjust_holdings_response(response, scope, portfolio_code)

data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object


/home/jovyan/sample-notebooks/examples/use-cases/cocoon.py:564: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_type_update_map[data_type_lusid], copy=False)


Check for missing holding properties complete


Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_8
Code: Portfolio-Z
Adjusted Holdings Effective From: 2019-02-07 00:00:00+00:00
Adjusted Holdings Created On: 2019-07-30 09:21:12.379221+00:00

data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object
Check for missing holding properties complete


Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_8
Code: Portfolio-Z
Adjusted Holdings Effective From: 2019-02-07 00:00:00+00:00
Adjusted Holdings Created On: 2019-07-30 09:21:13.730320+00:00

data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object
Check for missing holding properties complete


Holdings Successfully Adjusted for Portfolio
Scope: my_test_scope_8
Code: Portfolio-Z
Adjusted Holdings Effective From: 2019-02-07 00:00:00+00:00
Adjusted Holdi

In [15]:
response = client.transaction_portfolios.get_holdings(
    scope=scope,
    code="Portfolio-X",
    property_keys=["Instrument/default/Name"])

holdings = prettyprint.get_holdings_df(response)
holdings

,Holding/my_test_scope_8/BaseMarketValue,Holding/my_test_scope_8/Buy_Currency,Holding/my_test_scope_8/EffectiveDate,Holding/my_test_scope_8/FundCode,Holding/my_test_scope_8/ISINSecurityIdentifier,Holding/my_test_scope_8/LocalCurrencyCode,Holding/my_test_scope_8/LocalMarketValue,Holding/my_test_scope_8/LocalPrice,Holding/my_test_scope_8/PrimeBroker,Holding/my_test_scope_8/Quantity,...,Instrument/default/Name,_holding_type,_instrument_uid,_settled_units,_transaction,_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,discriminator
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tesla Inc,P,LUID_S9BYYAL3,286263.0,None,286263.0,66115302.0,USD,66115302.0,None
1,22038434.0,USD,24/11/2018,Portfolio-X,US88160R1014,USD,22038434.0,230.96,MS,95421.0,...,Tesla Inc,P,LUID_S9BYYAL3,95421.0,None,95421.0,0.0,ZZZ,0.0,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sainsbury,P,LUID_UWJVIUUD,2827062.0,None,2827062.0,566825931.0,GBP,566825931.0,None
3,188941977.0,GBP,05/02/2019,Portfolio-X,XS1139087933,GBX,188941977.0,200.50,BAML,942354.0,...,Sainsbury,P,LUID_UWJVIUUD,942354.0,None,942354.0,0.0,ZZZ,0.0,None


In [16]:
holdings.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 23 columns):
Holding/my_test_scope_8/BaseMarketValue            2 non-null float64
Holding/my_test_scope_8/Buy_Currency               2 non-null object
Holding/my_test_scope_8/EffectiveDate              2 non-null object
Holding/my_test_scope_8/FundCode                   2 non-null object
Holding/my_test_scope_8/ISINSecurityIdentifier     2 non-null object
Holding/my_test_scope_8/LocalCurrencyCode          2 non-null object
Holding/my_test_scope_8/LocalMarketValue           2 non-null float64
Holding/my_test_scope_8/LocalPrice                 2 non-null float64
Holding/my_test_scope_8/PrimeBroker                2 non-null object
Holding/my_test_scope_8/Quantity                   2 non-null float64
Holding/my_test_scope_8/SEDOLSecurityIdentifier    2 non-null object
Holding/my_test_scope_8/SecurityDescription        2 non-null object
Holding/my_test_scope_8/is_cash_with_currency      2 non-null objec

# 4) Add your transactions

## a) Import transactions data

In [17]:
transactions_data = positions_file.copy(deep=True)
transactions_data

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [18]:
transactions_data['transaction_type'] = 'FundsIn'
transactions_data['exchange_rate'] = 1
transactions_data['is_cash_with_currency'] = np.NaN
transactions_data['Local Currency Code'] = transactions_data['Local Currency Code'].apply(
    lambda x: "GBp" if x == "GBX" else x)
transactions_data['transaction_id'] = [str(uuid.uuid4()) for _ in range(len(transactions_data.index))]
transactions_data['Effective Date'] = transactions_data['Effective Date'].apply(
    lambda x: pytz.utc.localize(x))
transactions_data.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency,transaction_type,exchange_rate,is_cash_with_currency,transaction_id
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP,FundsIn,1,NaN,00527843-7995-4124-b001-a12927215514
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD,FundsIn,1,NaN,26223303-fb08-4b2e-83d0-bf8aa35b3bc8
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD,FundsIn,1,NaN,25b5db39-adc5-40bf-9a7b-c036f7f490eb
3,Portfolio-X,05/02/2019,BAML,GBp,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP,FundsIn,1,NaN,7b1f2c44-3ed5-4a39-b539-75e808639ef2
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD,FundsIn,1,NaN,6cdbfecd-867d-4aa6-8695-aecb3996e766


## c) Set your transactions field mapping

In [19]:
transaction_field_mapping_required = {
    "portfolio_code": "FundCode",
    "transaction_id": "transaction_id",
    "transaction_type": 'transaction_type',
    "transaction_date": 'Effective Date',
    "settlement_date": 'Effective Date',
    "units": "Quantity",
    "transaction_price.price": "Local Price",
    "total_consideration.amount": "Local Market Value",
    "total_consideration.currency": "Local Currency Code",
    "transaction_currency": "Local Currency Code"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "exchange_rate"
}

transaction_identifier_mapping = {
  'identifier_mapping': {
      'Isin': 'ISIN Security Identifier',
      'Sedol': 'SEDOL Security Identifier'
  },
  'is_cash_with_currency': 'is_cash_with_currency'
}

## d) Resolve to your instrument master

In [20]:
transactions_data = cocoon.resolve_instruments(
    client=client,
    data_frame=transactions_data,
    identifier_mapping=holding_identifier_mapping)

current_time = datetime.now(pytz.UTC)

transactions_data.loc[
    transactions_data['resolvable'] == False].to_csv(
        './data/UnResolved{}-{}-{}-{}-{}-{}.csv'.format(
            'Transactions',
            current_time.year,
            current_time.month,
            current_time.day,
            current_time.hour,
            current_time.minute))

transactions_data['resolvable'].value_counts()

Up to row 0


True    5
Name: resolvable, dtype: int64

## e) Add your transactions

In [21]:
transactions_data = transactions_data.loc[transactions_data['resolvable'] == True]

responses = cocoon.load_file_multiple_portfolios(
    client=client, 
    scope=scope, 
    data_frame=transactions_data,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    source='Client',
    file_type='transaction')

for portfolio_code, response in responses.items():
    print ('\n')
    prettyprint.transactions_response(response, scope, portfolio_code)

data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object
Check for missing transaction properties complete


Transactions Successfully Upserted into Portfolio
Scope: my_test_scope_8
Code: Portfolio-Z
Transactions Effective From: 2019-02-07 00:00:00+00:00
Transactions Created On: 2019-07-30 09:21:19.338078+00:00



Transactions Successfully Upserted into Portfolio
Scope: my_test_scope_8
Code: Portfolio-X
Transactions Effective From: 2019-05-02 00:00:00+00:00
Transactions Created On: 2019-07-30 09:21:19.741093+00:00



In [22]:
response = client.transaction_portfolios.get_holdings(
    scope=scope,
    code="Portfolio-X",
    property_keys=["Instrument/default/Name"])

holdings = prettyprint.get_holdings_df(response)
holdings

,Holding/my_test_scope_8/BaseMarketValue,Holding/my_test_scope_8/Buy_Currency,Holding/my_test_scope_8/EffectiveDate,Holding/my_test_scope_8/FundCode,Holding/my_test_scope_8/ISINSecurityIdentifier,Holding/my_test_scope_8/LocalCurrencyCode,Holding/my_test_scope_8/LocalMarketValue,Holding/my_test_scope_8/LocalPrice,Holding/my_test_scope_8/PrimeBroker,Holding/my_test_scope_8/Quantity,...,Instrument/default/Name,_holding_type,_instrument_uid,_settled_units,_transaction,_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,discriminator
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tesla Inc,P,LUID_S9BYYAL3,381684.0,None,381684.0,88153736.0,USD,88153736.0,None
1,22038434.0,USD,24/11/2018,Portfolio-X,US88160R1014,USD,22038434.0,230.96,MS,95421.0,...,Tesla Inc,P,LUID_S9BYYAL3,95421.0,None,95421.0,0.0,ZZZ,0.0,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sainsbury,P,LUID_UWJVIUUD,3769416.0,None,3769416.0,755767908.0,GBP,755767908.0,None
3,188941977.0,GBP,05/02/2019,Portfolio-X,XS1139087933,GBX,188941977.0,200.50,BAML,942354.0,...,Sainsbury,P,LUID_UWJVIUUD,942354.0,None,942354.0,0.0,ZZZ,0.0,None
